https://cloud.google.com/vision/automl/docs

!sudo pip3 install google-cloud-automl

1. CREATING DATASET
2. Import data items into the dataset
3. Label the items

In [ ]:
import os
from google.cloud import automl

In [ ]:
!gsutil cat -r 0-500 'gs://flr_cls/csv/all_data.csv' 

In [ ]:
####################
## Initialize Variables
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "sa-neophilex-automl-admin.json"
project_id = "neophilex-alpha"
dataset_id = 'ICN5335346088922578944'  ## this will keep changing everytime you create a new one
dataset_name = 'flower_classification'
csv_path = 'gs://flr_cls/csv/all_data.csv'  ## please check the name of bucket
model_id = 'ICN2194557039319973888'   ## this will keep changing everytime you create a new one
model_display_name = 'flr_cls_m1_v1'
file_path = '01_input/flowers/roses-001.jpg'  ## location of the image you want to predict



client = automl.AutoMlClient()      ## if GOOGLE_APPLICATION_CREDENTIALS is not defined then you will have to use from file

project_location = client.location_path(project_id, "us-central1")

## list datasets

https://pyformat.info/

In [ ]:
for dataset in client.list_datasets(project_location):
    print("{:<25}  \t {:<25}".format(dataset.display_name, dataset.name))

## Create DATASET

In [ ]:
# Specify the classification type
# Types:
# MultiLabel: Multiple labels are allowed for one example.
# MultiClass: At most one label is allowed per example.
# https://cloud.google.com/automl/docs/reference/rpc/google.cloud.automl.v1#classificationtype
metadata = automl.types.ImageClassificationDatasetMetadata(
    classification_type=automl.enums.ClassificationType.MULTILABEL
)

dataset = automl.types.Dataset(
    display_name=dataset_name,
    image_classification_dataset_metadata=metadata,
)

# Create a dataset with the dataset metadata in the region.
response = client.create_dataset(project_location, dataset)

created_dataset = response.result()

# Display the dataset information
print("Dataset name: {}".format(created_dataset.name))
print("Dataset id: {}".format(created_dataset.name.split("/")[-1]))

## Importing items into a dataset

In [ ]:
# Get the full path of the dataset.
dataset_full_id = client.dataset_path(project_id, "us-central1", dataset_id)

# Get the multiple Google Cloud Storage URIs
input_uris = csv_path.split(",")
print(input_uris)
input()
gcs_source = automl.types.GcsSource(input_uris=input_uris)
input_config = automl.types.InputConfig(gcs_source=gcs_source)

print(input_config)
input()

# Import data from the input URI
response = client.import_data(dataset_full_id, input_config)

print("Processing import...")
print("Data imported. {}".format(response.result()))

## Create a model

In [ ]:
# Leave model unset to use the default base model provided by Google
# train_budget_milli_node_hours: The actual train_cost will be equal or
# less than this value.
# https://cloud.google.com/automl/docs/reference/rpc/google.cloud.automl.v1#imageclassificationmodelmetadata
metadata = automl.types.ImageClassificationModelMetadata(train_budget_milli_node_hours=8000)

model = automl.types.Model(
    display_name=model_display_name,
    dataset_id=dataset_id,
    image_classification_model_metadata=metadata,
)

# Create a model with the model metadata in the region.
response = client.create_model(project_location, model)

print("Training operation name: {}".format(response.operation.name))
print("Training started...")

## list operations

In [ ]:
response = client.transport._operations_client.list_operations(project_location, "")

print("List of operations:")
for operation in response:
    print("Name: {}".format(operation.name))
    print("Operation details:")
    print(operation)

## Getting the status of an operation

In [ ]:
# TODO(developer): Uncomment and set the following variables
# operation_full_id = \
#     "projects/[projectId]/locations/us-central1/operations/[operationId]"

response = client.transport._operations_client.get_operation(
    'projects/740070380126/locations/us-central1/operations/IOD4430860885879160832'
)

print("Name: {}".format(response.name))
print("Operation details:")
print(response)

## list models

In [ ]:
for model in client.list_models(project_location):
    print("{:25} {:75} {:1}".format(model.display_name, model.name, model.deployment_state))

In [ ]:
##choose the correct model for further processing
model_id = '<assign the id>'

## Getting information about a model

In [ ]:
model_id = "ICN2194557039319973888"

# Get the full path of the model.
model_full_id = client.model_path(project_id, "us-central1", model_id)
model = client.get_model(model_full_id)

# Retrieve deployment state.
if model.deployment_state == automl.enums.Model.DeploymentState.DEPLOYED:
    deployment_state = "deployed"
else:
    deployment_state = "undeployed"

# Display the model information.
print("Model name: {}".format(model.name))
print("Model id: {}".format(model.name.split("/")[-1]))
print("Model display name: {}".format(model.display_name))
print("Model create time:")
print("\tseconds: {}".format(model.create_time.seconds))
print("\tnanos: {}".format(model.create_time.nanos))
print("Model deployment state: {}".format(deployment_state))

## Deploy Model

In [ ]:
# Get the full path of the model.
model_full_id = client.model_path(project_id, "us-central1", model_id)
response = client.deploy_model(model_full_id)

print("Model deployment finished. {}".format(response.result()))

## Update a model's node number

In [ ]:
from google.cloud import automl

# TODO(developer): Uncomment and set the following variables
# project_id = "YOUR_PROJECT_ID"
# model_id = "YOUR_MODEL_ID"

client = automl.AutoMlClient()
# Get the full path of the model.
model_full_id = client.model_path(project_id, "us-central1", model_id)

# node count determines the number of nodes to deploy the model on.
# https://cloud.google.com/automl/docs/reference/rpc/google.cloud.automl.v1#imageclassificationmodeldeploymentmetadata
metadata = automl.types.ImageClassificationModelDeploymentMetadata(
    node_count=2
)
response = client.deploy_model(
    model_full_id, image_classification_model_deployment_metadata=metadata
)

print("Model deployment finished. {}".format(response.result()))

## Online (individual) prediction

In [ ]:
from google.cloud import automl

# TODO(developer): Uncomment and set the following variables
project_id = "neophilex-alpha"
model_id = "ICN2194557039319973888"
file_path = "01_input/flowers/roses-001.jpg"

prediction_client = automl.PredictionServiceClient()

# Get the full path of the model.
model_full_id = prediction_client.model_path(project_id, "us-central1", model_id)

# Read the file.
with open(file_path, "rb") as content_file:
    content = content_file.read()

image = automl.types.Image(image_bytes=content)
payload = automl.types.ExamplePayload(image=image)

# params is additional domain-specific parameters.
# score_threshold is used to filter the result
# https://cloud.google.com/automl/docs/reference/rpc/google.cloud.automl.v1#predictrequest
params = {"score_threshold": "0.8"}

response = prediction_client.predict(model_full_id, payload, params)
print("Prediction results:")
for result in response.payload:
    print("Predicted class name: {}".format(result.display_name))
    print("Predicted class score: {}".format(result.classification.score))

## Undeploy Model

In [ ]:
response = client.undeploy_model(model_full_id)
print("Model undeployment finished. {}".format(response.result()))